In [1]:
##### Import package###
import numpy as np
import matplotlib.pylab as plt
from scipy.integrate import odeint
from numpy.linalg import norm
from pylab import *
import pylab as p
from scipy import integrate
from IPython import get_ipython, display
import ipywidgets as widgets
from ipywidgets import (
    interactive,
    interact,
    fixed,
    FloatRangeSlider,
    FloatSlider,
    interactive_output,
    interact_manual,
    Button,
    ToggleButton

)
from google.collab import files
%matplotlib inline
#matplotlib.rcParams['text.usetex'] = True

###################################
#####Main function######

LTrajectory = []  # Defining the list of trajectories


def LotKaVoltInteract(K1, K2, r1, alpha21, r2, alpha12, Vectors, Xo_abs, Yo_abs):
    global X, fig, Block, X0
    # The two function defining the system and the system

    def f(x, y):
        f = r1*x*(1-(x+alpha21*y)/K1)
        return f

    def g(x, y):
        g = r2*y*(1-(y+alpha12*x)/K2)
        return g

    def dX_dt(X, t=0):
        return array([f(X[0], X[1]), g(X[0], X[1])])
 # Maximal value of the two coordinates
    xmax = 1.05*max(K1, K2/alpha12) if alpha12 != 0 else 1.05*K1
    ymax = 1.05*max(K2, K1/alpha21) if alpha21 != 0 else 1.05*K2

    fig = plt.figure(figsize=(11, 11))  # The figure
    p.title('Espace des phases, isoclines, points fixes et trajectoires')
    plt.xlabel('Population 1')
    plt.ylabel('Population 2')
    plt.xlim(-0.05*xmax, xmax)
    plt.ylim(-0.05*ymax, ymax)
    ########## The vector field ##########
    if Vectors == True:
        nb_points = 15
        x = np.linspace(0, xmax, nb_points)
        y = np.linspace(0, ymax, nb_points)
        X1, Y1 = meshgrid(x, y)
        DX1, DY1 = dX_dt([X1, Y1])
        M = (hypot(DX1, DY1))
        M[M == 0] = 1.
        DX1 /= M
        DY1 /= M
        Q = p.quiver(X1, Y1, DX1, DY1, M, cmap=p.cm.jet)

    # the two isoclines
    plt.plot([K1, 0], [0, K1/alpha21],
             'red') if alpha21 != 0 else plt.plot([K1, K1], [0, ymax], 'red')
    plt.plot([0, 0], [0, ymax], 'red')
    plt.plot([K2/alpha12, 0], [0, K2],
             'green') if alpha12 != 0 else plt.plot([0, xmax], [K2, K2], 'green')
    plt.plot([0, xmax], [0, 0], 'green')

    # The points of equlibrium
    plt.scatter(0, 0, c='black', s=80)
    if 1-alpha12*alpha21 != 0:
        xinter = (K1-alpha21*K2)/(1-alpha12*alpha21)
        yinter = (K2-alpha12*K1)/(1-alpha12*alpha21)
        if xinter >= 0 and yinter >= 0:
            plt.scatter(xinter, yinter, c='black', s=80)
    # The capacities
    plt.scatter(0, K2, c='black', s=80)
    plt.scatter(K1, 0, c='black', s=80)
    # Initial state
    Xo = Xo_abs*xmax/100
    Yo = Yo_abs*ymax/100
    plt.scatter(Xo, Yo, c='red', s=30)
    ####################

    ######## xticks, yticks
    if alpha12 != 0:
        plt.xticks([0, K1, K2/alpha12], ['0', r'$K_1$',
                   r'$K_2/\alpha_{12}$']) if K1 != K2/alpha12 else plt.xticks([0, K1], ['0', r'$K_1=K_2/\alpha_{12}$'])
    if alpha21 != 0:
        plt.yticks([0, K2, K1/alpha21], ['0', r'$K_2$',
                   r'$K_1/\alpha_{21}$']) if K2 != K1/alpha21 else plt.yticks([0, K2], ['0', r'$K_2=K_1/\alpha_{21}$'])

    if alpha12 == 0:
        plt.xticks([0, K1], ['0', r'$K_1$'])
    if alpha21 == 0:
        plt.yticks([0, K2], ['0', r'$K_2$'])

    # Computing and drawing a trajectory
    t = linspace(0, 50,  500)
    X0 = array([Xo, Yo])
    X = integrate.odeint(dX_dt, X0, t)
    plt.plot(X[:, 0], X[:, 1], lw=1.5)

    # Drawing trajectories in the list
    for T in LTrajectory:
        plt.plot(T[0], T[1], lw=2)


#######################
# Defining the widgets and Layout
#################################

# Common layout of button and slider
curseur1Layout = widgets.Layout(width='300px', height='30px')
buttonLayout = widgets.Layout(width='150px', height='30px')
# Defining Slider and button
# Slider for systeme parameters

# init
Xo_abs = widgets.IntSlider(min=0, max=100, value=98, description=r'$x_0$', readout=False,
                           continuous_update=False, layout=widgets.Layout(width='700px', height='15px'))
Yo_abs = widgets.IntSlider(min=0, max=100, value=98, description=r'$y_0$', orientation='vertical',
                           readout=False, continuous_update=False, layout=widgets.Layout(width='15px', height='650px'))
# Buttons

K1 = widgets.FloatSlider(min=1, max=10, value=3,
                         description=r'$K_1$', layout=curseur1Layout)
K2 = widgets.FloatSlider(min=1, max=10, value=3,
                         description=r'$K_2$', layout=curseur1Layout)
r1 = widgets.FloatSlider(min=0.1, max=5, value=2,
                         description=r'$r_1$', layout=curseur1Layout)
r2 = widgets.FloatSlider(min=0.1, max=5, value=1,
                         description=r'$r_2$', layout=curseur1Layout)
alpha21 = widgets.FloatSlider(
    min=0, max=10, value=0.6, step=0.01, description=r'$\alpha_{21}$', layout=curseur1Layout)
alpha12 = widgets.FloatSlider(
    min=0, max=10, value=0.5, step=0.01, description=r'$\alpha_{12}$', layout=curseur1Layout)
Vectors = widgets.ToggleButton(
    value=True, description='Click for hide vectors', button_style='info', layout=buttonLayout)
keep = Button(description="keep trajectory",
              button_style='info', layout=buttonLayout)
EraseAll = Button(description="Erase all",
                  button_style='info', layout=buttonLayout)
EraseLast = Button(description="Erase last trajectory",
                   button_style='info', layout=buttonLayout)
Save = Button(description="Save figure",
              button_style='info', layout=buttonLayout)
#Fix= Button(description="Fix",button_style='info',layout=buttonLayout)
Release = Button(description="Unfreeze Cursors.",
                 button_style='info', layout=buttonLayout)
##############
# Function on click


def on_keep_clicked(_):
    global LTrajectory, K1, K2, alpha12, alpha21, r1, r2, u
    LTrajectory.append([X[:, 0], X[:, 1]])
    for u in [K1, K2, alpha12, alpha21, r1, r2]:
        u.disabled = True


def on_erase_clicked(_):
    global LTrajectory
    LTrajectory = []
    Xo_abs.value = Xo_abs.value+1


def last_erase_clicked(_):
    global LTrajectory
    if len(LTrajectory) > 0:
        LTrajectory = LTrajectory[:len(LTrajectory)-1]
    Xo_abs.value = Xo_abs.value+1


def SaveTheFigure(_):
    fig.savefig('fig.pdf', dpi=600)
    files.download('fig.pdf')


def Block_Sliders(_):
    global K1, K2, alpha12, alpha21, r1, r2, u
    for u in [K1, K2, alpha12, alpha21, r1, r2]:
        u.disabled = True


def Release_Sliders(_):
    global K1, K2, alpha12, alpha21, r1, r2, u
    for u in [K1, K2, alpha12, alpha21, r1, r2]:
        u.disabled = False


# Fix.on_click(Block_Sliders)
Release.on_click(Release_Sliders)
# Fix.observe(Block_Sliders,names='value')
# Actions on click
keep.on_click(on_keep_clicked)
EraseLast.on_click(last_erase_clicked)
EraseAll.on_click(on_erase_clicked)
Save.on_click(SaveTheFigure)
# Ouput the figure
out = widgets.interactive_output(LotKaVoltInteract, {'K1': K1, 'K2': K2, 'r1': r1, 'r2': r2,
                                                     'alpha21': alpha21, 'alpha12': alpha12,
                                                     'Vectors': Vectors, 'Xo_abs': Xo_abs, 'Yo_abs': Yo_abs})
# Panel control
ui_Wid_left = widgets.VBox([K1, K2, r1, r2, alpha21, alpha12, widgets.HBox([Vectors, Release]), widgets.HBox([keep, EraseLast]),
                            widgets.HBox([EraseAll, Save])])
# The final layout

grid = widgets.GridspecLayout(15, 15)
grid[0:13, 0] = Yo_abs
grid[0:13, 1:9] = out
grid[0:13, 10:] = ui_Wid_left
grid[13:, 1:8] = Xo_abs
grid

GridspecLayout(children=(IntSlider(value=98, continuous_update=False, description='$y_0$', layout=Layout(grid_…

## Le modèle de Lotka-Volterra compétitif
Ce notebook permet de visualiser l'espace des phases et les trajectoires du modèle de Lotka-Volterra compétitif. Rappelons que ce modèle est un système d'équations différentielles linéaires modélisant l'évolution de deux populations en interaction :

$$
\left\{\begin{matrix}
\displaystyle x'=r_1x\left(1-\frac{x+\alpha_{21} y}{K_1}\right)\\
\\
\displaystyle y'= r_2y\left(1-\frac{y+\alpha_{12}x}{K_2}\right).
\end{matrix}
\right.
$$

Les deux populations suivent un modèle logistique de constante $(r_1,K_1)$  pour la première population dont la taille est donnée par la fonction $x$ et  $(r_2,K_2)$ pour la deuxième population dont la taille est donnée par la fonction $y$. Ces constantes sont ici supposées strictement positives. Les deux constantes $\alpha_{21}$ et $\alpha_{12}$ mesurent l'interaction d'une espèce sur l'autre. Dans ce notebook les interactions sont défavorables aux deux espèces, c'est à dire que les coefficients $\alpha_{21}$ et $\alpha_{12}$ sont positifs ou nuls. 

Par défaut, la figure affiche les isoclines, les points fixes, les vecteurs tangents et une trajectoire dont l'état initial est marqué par un point rouge. En cliquant sur le bouton *Click for hide vectors* on peut cacher les vecteurs tangents aux trajectoires. 
On peut avec l'aide des curseurs faire varier tous les paramètres du système ainsi que l'état initial. Un fois que l'on a obtenu un système qui nous convient, on peut enregister en mémoire les trajectoires avec le bouton adéquat. Attention : une fois qu'une trajectoire est enregistrée, les curseurs modifiant le système sont désactivés, ils peuvent être réactivés par le bouton *Unfreeze Cursors*.  On peut avec deux autres boutons effacer la dernière trajectoire ou bien toutes les trajectoires. Et pour finir, en cliquant sur le bouton *Save the figure*, on peut enregistrer la figure, on trouvera la figure sous format .pdf dans le répetoire à partir duquel a été lancé le notebook. Les couleurs des vecteurs sont reliées à la *vitesse* des trajectoires, par contre les couleurs des trajectoires sont purement décoratives. 

Le point de départ de ce notebook est un programme statique concernant le système Lotka-Volterra classique disponible sur ce [site](http://scipy.github.io/old-wiki/pages/Cookbook/LoktaVolterraTutorial). 
Je remercie Tanguy Lefort pour sa disponibilité et ses conseils sur le package *Interact*. 